In [ ]:
!pip install pandas
!pip install xlrd
!pip install geopandas

In [3]:
import pandas as pd
import geopandas
from geopandas import GeoDataFrame as gpd
import numpy as np
import json

# 1. Carbon Flux to the Atmosphere from Land-Use Changes 1850-2005 Dataset

In [4]:
usecols=['Year','Global','USA','Canada','S+C America','Europe','Trop.Africa','China','S+SE Asia','Pac.Dev.Reg']
landuse = pd.read_excel("../data/Global_land-use_flux-1850_2005.xls"
                        , sheet_name="net fluxes"
                        , usecols=usecols)
landuse.head()

,Year,Global,USA,Canada,S+C America,Europe,Trop.Africa,China,S+SE Asia,Pac.Dev.Reg
0,1850,500.6,164.0922,5.5476,23.4757,55.0441,-1.3484,101.8392,87.3469,2.0458
1,1851,492.7,165.7256,5.3626,23.1520,55.0156,-1.1192,93.0766,86.9100,2.0419
2,1852,548.5,230.6725,5.3380,22.8618,54.9874,-1.0033,83.8307,86.9385,2.0379
3,1853,546.8,238.5149,5.3138,22.6017,54.9590,-1.0513,74.2140,86.9935,2.0338
4,1854,544.8,246.1846,5.2899,22.3687,54.9304,-0.9855,64.3129,87.0630,2.0297


In [5]:
landuse.columns=['date','Global','USA','Canada','America','Europe','Africa','China','Asia','PacificRegion']
landuse.head()

,date,Global,USA,Canada,America,Europe,Africa,China,Asia,PacificRegion
0,1850,500.6,164.0922,5.5476,23.4757,55.0441,-1.3484,101.8392,87.3469,2.0458
1,1851,492.7,165.7256,5.3626,23.1520,55.0156,-1.1192,93.0766,86.9100,2.0419
2,1852,548.5,230.6725,5.3380,22.8618,54.9874,-1.0033,83.8307,86.9385,2.0379
3,1853,546.8,238.5149,5.3138,22.6017,54.9590,-1.0513,74.2140,86.9935,2.0338
4,1854,544.8,246.1846,5.2899,22.3687,54.9304,-0.9855,64.3129,87.0630,2.0297


In [6]:
continents = ['Global','America','Africa','Asia','PacificRegion','Europe']
dict_country={}
ls_emission_country = []
dict_final = {}
# interate the emision of each country in the dataframe
for column in landuse.columns:
    if(column is 'date'):
        continue
    emission = landuse[['date', column]]
    ## dict 1: geocomponents
    dict_geo={}
    if (column not in continents):
        dict_geo["scale_name"]="country"
    else:
        dict_geo["scale_name"]="continent"
    dict_geo["geoComponent_name"]=column
    if (len(column) == 3):# FRA
        dict_geo["iso_code"]={"alpha2":'None',"alpha3":column}
    else:
        dict_geo["iso_code"]={"alpha2":column,"alpha3":'None'}        
    dict_geo["properties"]={"area":'None'}
    # dict2: emission
    # rename columns
    emission.columns = ['date', 'value']
    # cast all numerical values into string
    emission = emission.applymap(repr)
    emission['sector_name'] = ['Land Use']*len(emission)
    emission['gas_name'] = ['CO2']*len(emission)
    emission['Unit'] = ['NA']*len(emission)
    emission['data_source_name'] = ['CDIAC']*len(emission)
    emission['subsector_name'] = ['gas_co2']*len(emission)

    ls_emission_country.append({"geoComponent": dict_geo, "emissions": emission.to_dict('record')})
    dict_final.update({len(ls_emission_country)-1: ls_emission_country})
    
with open("../data/landuse.json", "w") as outfile: 
    json.dump(dict_final, outfile)

/srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/frame.py:1554: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
